<a href="https://colab.research.google.com/github/ThunderCold/Brain-Computer-Interface/blob/main/Machine%20Learning/20221113_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 準備訓練資料與驗證資料

In [ ]:
# 匯入必要模組
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from google.colab import drive

In [ ]:
# 將 Google Colab 連線至 Google Drive
drive.mount('/content/drive')

In [ ]:
# 讀入資料集
df = pd.read_csv('/content/drive/MyDrive/ML/Train_data.csv')
df_test = pd.read_csv('/content/drive/MyDrive/ML/Test_data.csv')

In [ ]:
# 移除缺失值
for i in df.columns:
    df = df.drop(index=df[df[i].isnull()].index)

for i in df_test.columns:
    df_test=df_test.drop(index=df_test[df_test[i].isnull()].index)

In [ ]:
# 轉換非數字型態之特徵值
df["gender"] = df["gender"].map({'M':1, 'F':0})
df_test["gender"]=df_test["gender"].map({'M':1, 'F':0})

le = preprocessing.LabelEncoder()
le.fit(['A','B','C','D'])
df['class']=le.transform(df['class'])

In [ ]:
# 分割輸入與輸出值
x = df.iloc[:, :-1]
y = df.iloc[:, -1]
y = y.astype('int')

# 分割 train data 與 test data
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=0.2)

In [ ]:
# 將資料進行標準化
minmax = preprocessing.MinMaxScaler()

x_train = minmax.fit_transform(x_train)
x_validation = minmax.transform(x_validation)
df_test = minmax.transform(df_test)

## 訓練機器學習模型（Dicision Tree）

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# 建立模型並設置參數
model = DecisionTreeClassifier(max_depth=10)

# 帶入資料並訓練
model.fit(x_train, y_train)

print(f'Accuracy on validation set is {model.score(x_validation, y_validation)}')

## 訓練機器學習模型（Random Forest）

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 建立模型並設置參數
model = RandomForestClassifier(n_estimators=500,random_state=0,max_depth=10)

# 帶入資料並訓練
model.fit(x_train, y_train)

print(f'Accuracy on validation set is {model.score(x_validation, y_validation)}')

## 輸出結果

In [ ]:
# 產生預測結果
data_id = range(1,2041)
level = model.predict(df_test)

# 產生資料集最終結果
mydict = {"data_id":data_id, "level":level}
df_output = pd.DataFrame(mydict)

# 將數字特徵值轉換回文字型態
for i in df_output["level"].index:
    if df_output["level"][i] == 0:
        df_output["level"][i] = 'A'
    if df_output["level"][i] == 1:
        df_output["level"][i] = 'B'
    if df_output["level"][i] == 2:
        df_output["level"][i] = 'C'
    if df_output["level"][i] == 3:
        df_output["level"][i] = 'D'

# 輸出為 CSV 檔
df_output.to_csv()